https://scryfall.com/docs/api/bulk-data

Oracle Cards

Problem: Prices are the price of the latest card version
(if latest version is only digital, None prices)
Solve problem with bridge script to update data?

In [1]:
import ujson as json
import os

from tqdm.auto import tqdm
from utils import NEW_DATA_FOLDER, SCRYFALL_FOLDER, is_mtga_legal, save_card_to_dict

Copied files


In [2]:
with open(os.path.join(SCRYFALL_FOLDER, "oracle-cards.json"), encoding="utf-8") as j:
    data = json.load(j)

In [3]:
for card in data:
    print(card["name"])
    print(card["rarity"].title())
    print(card["prices"])
    break

Static Orb
Rare
{'usd': '21.59', 'usd_foil': '419.85', 'usd_etched': None, 'eur': '13.33', 'eur_foil': '250.00', 'tix': '0.52'}


In [4]:
rarities = dict()
card_types = dict()
eur_prices = dict()
usd_prices = dict()
mtgo_prices = dict()

In [5]:
for card in tqdm(data):
    lower_name = card["name"].lower()

    card_type = card["type_line"].split(" —")[0].split(" /")[0]
    if card_type != "Card":
        save_card_to_dict(lower_name, card_type, card_types)

    if is_mtga_legal(card):
        # rarities[card["name"]] = card["rarity"].title()
        save_card_to_dict(card["name"], card["rarity"].title(), rarities)

    for currency, dictionary in {
        "usd": usd_prices,
        "eur": eur_prices,
        "tix": mtgo_prices,
    }.items():
        try:
            save_card_to_dict(lower_name, float(card["prices"][currency]), dictionary)
        except TypeError:  # price is None
            pass

# correct basic Lands
rarities.update(
    {
        "Forest": "Basic Land",
        "Swamp": "Basic Land",
        "Mountain": "Basic Land",
        "Island": "Basic Land",
        "Plains": "Basic Land",
    }
)

  0%|          | 0/26671 [00:00<?, ?it/s]

In [6]:
rarities["Forest"]

'Basic Land'

### Save Rarities

In [7]:
with open(os.path.join(NEW_DATA_FOLDER, "new_rarities.py"), "w", encoding="utf-8") as f:
    f.write("rarity = " + str(rarities))

### Types

In [8]:
card_types["valentin, dean of the vein"]

'Legendary Creature'

In [9]:
corrections = {
    "prismari command": "Instant",
    "valentin, dean of the vein": "Legendary Creature",
    "arrogant poet": "Creature",
    "ephemerate": "Instant",
}

for k, v in corrections.items():
    if card_types[k] != v:
        print(k, "was", card_types[k])
        card_types[k] = v
        print(k, "Corrected")

In [10]:
print(card_types["liliana of the veil"])
print(card_types["lorehold command"])

Legendary Planeswalker
Instant


### Save Card Types

In [11]:
with open(
    os.path.join(NEW_DATA_FOLDER, "new_card_types.py"), "w", encoding="utf-8"
) as f:
    f.write("cards_to_types = " + str(card_types))

In [12]:
print(eur_prices["liliana of the veil"])
print(mtgo_prices["black lotus"])
print(eur_prices["urza's saga"])
print(eur_prices["volcanic island"])

35.94
11.34
35.15


KeyError: 'volcanic island'

In [13]:
print(eur_prices["test of talents"])

0.05


### Save Prices

In [14]:
for currency, dictionary in {
    "usd_prices": usd_prices,
    "eur_prices": eur_prices,
    "mtgo_prices": mtgo_prices,
}.items():
    file_name = "new_" + currency
    with open(
        f"{os.path.join(NEW_DATA_FOLDER, file_name)}.py", "w", encoding="utf-8"
    ) as f:
        f.write(f"{currency} = {dictionary}")